<b>Name: Rohit Garla</b>

<b>Net ID:rg3365</b>

<b>Name: Mengrui Zhang</b>

<b>Net ID:mz2258</b>

# <center><b>Assignment 2</b>

In [1]:
df = spark.read.parquet("/Users/garlarohit/desktop/Assignment 1/mortgage_sample")
df.show()
df.createOrReplaceTempView("data")
spark.sql("select count(FICO) from data").show()

AnalysisException: 'Path does not exist: file:/Users/garlarohit/desktop/Assignment 1/mortgage_sample;'

In [157]:
from pyspark.mllib.util import MLUtils
from pyspark.sql import Row
import math
from pyspark.ml.classification import RandomForestClassifier
from pyspark.sql.functions import lit
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.evaluation import RegressionMetrics

In [152]:
df2 = df.limit(10000)
df.groupBy('label').count().show()
df2.groupBy('label').count().show()
fractions = {1: 1, 2: 0.2, 3: 0.2}
df3 = df.stat.sampleBy("label", fractions, 12).limit(10000)
df3.groupBy('label').count().show()
fractions = {1: 1, 2: 0.18, 3: 0.2}
df4 = df.stat.sampleBy("label", fractions, 12).limit(10000)
df4.groupBy('label').count().show()
fractions = {1: 1, 2: 0.16, 3: 0.2}
df5 = df.stat.sampleBy("label", fractions, 12).limit(10000)
df5.groupBy('label').count().show()
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=12,  maxDepth=10)
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=20)
model = rf.fit(df3)
model1 = rf.fit(df4)
lrModel = lr.fit(df5)
pred = model.transform(df3)
pred1 = model1.transform(df4)
pred2 = lrModel.transform(df5)
pred.show()

+-----+-------+
|label|  count|
+-----+-------+
|    1|  92996|
|    3|  27095|
|    2|2602038|
+-----+-------+

+-----+-----+
|label|count|
+-----+-----+
|    1|  350|
|    2| 9554|
|    3|   96|
+-----+-----+

+-----+-----+
|label|count|
+-----+-----+
|    1| 1532|
|    2| 8386|
|    3|   82|
+-----+-----+

+-----+-----+
|label|count|
+-----+-----+
|    1| 1666|
|    2| 8242|
|    3|   92|
+-----+-----+

+-----+-----+
|label|count|
+-----+-----+
|    1| 1864|
|    2| 8033|
|    3|  103|
+-----+-----+

+-----+--------+--------------------+----+--------------+--------+----------+------+-----+--------------------+--------------------+--------------------+----------+
| FICO|CLEANLTV|           INCENTIVE| AGE|FIRSTTIMEBUYER|INVESTOR|SECONDHOME|SEASON|label|            features|       rawPrediction|         probability|prediction|
+-----+--------+--------------------+----+--------------+--------+----------+------+-----+--------------------+--------------------+--------------------+--------

In [155]:
print("Multinomial coefficients: " + str(lrModel.coefficientMatrix))
print("Multinomial intercepts: " + str(lrModel.interceptVector))

Multinomial coefficients: DenseMatrix([[ -8.96215336e-04,  -1.56239975e-03,  -8.87493146e-01,
               -3.75400567e-04,  -5.20023376e-03,  -4.80593573e-03,
               -4.78263121e-03,  -2.61713366e-03],
             [  1.72915021e-03,  -1.32352183e-02,   3.76449456e+01,
               -9.17912597e-03,  -6.06413175e-02,  -3.99191287e-01,
               -1.64370725e-01,   1.59645654e-02],
             [  1.89525143e-03,  -7.31940466e-03,  -6.89399735e+01,
                1.88828396e-05,   2.43537855e-02,   2.76796727e-01,
                6.94222871e-02,   3.86378857e-03],
             [ -2.72818630e-03,   2.21170227e-02,   3.21825211e+01,
                9.53564370e-03,   4.14877658e-02,   1.27200496e-01,
                9.97310687e-02,  -1.72112203e-02]])
Multinomial intercepts: [-5.29636850861,2.13578062824,3.78207362107,-0.621485740706]


In [167]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Evaluate model
evaluator = MulticlassClassificationEvaluator()
evaluator.evaluate(pred2)

0.7189677764903939

In [176]:
evaluator = MulticlassClassificationEvaluator()

accuracy = evaluator.evaluate(pred1)
print("Test Error = " + str(1.0 - accuracy))

Test Error = 0.23577590805946735


In [177]:
evaluator = MulticlassClassificationEvaluator()

accuracy = evaluator.evaluate(pred)
print("Test Error = " + str(1.0 - accuracy))

Test Error = 0.22048781520941518


In [132]:
pred.rdd.map(lambda x : -math.log(x["probability"][int(x["prediction"])])).map(lambda x : Row(x)).toDF().show()

+--------------------+
|                  _1|
+--------------------+
| 0.49171341241220595|
|  0.3914155544992621|
| 0.35872024809906233|
| 0.33314678839709644|
|  0.2126039635323621|
| 0.04373437213380192|
|0.014663011479900548|
| 0.03899098057675509|
| 0.18011535121556285|
|  0.3718111356701752|
|  0.0907523236955305|
| 0.32437557559416624|
| 0.03288051902358533|
| 0.44533363122557135|
| 0.41225143178829127|
|0.016368581717375853|
| 0.14107539264398364|
| 0.08325769172184137|
| 0.39576793123603715|
|  0.2644348922827095|
+--------------------+
only showing top 20 rows



In [142]:
pf = pred.rdd.map(lambda x : -math.log(x["probability"][int(x["prediction"])])).map(lambda x : Row(x)).toDF()
pf1 = pred1.rdd.map(lambda x : -math.log(x["probability"][int(x["prediction"])])).map(lambda x : Row(x)).toDF()
pf2=pred2.rdd.map(lambda x : -math.log(x["probability"][int(x["prediction"])])).map(lambda x : Row(x)).toDF()

In [143]:
pf.printSchema()
pf.createOrReplaceTempView("entropy")
spark.sql("select avg(_1) from entropy").show()
pf1.createOrReplaceTempView("entropy")
spark.sql("select avg(_1) from entropy").show()
pf2.createOrReplaceTempView("entropy")
spark.sql("select avg(_1) from entropy").show()

root
 |-- _1: double (nullable = true)

+-------------------+
|            avg(_1)|
+-------------------+
|0.18846371663706205|
+-------------------+

+-------------------+
|            avg(_1)|
+-------------------+
|0.20423527885619078|
+-------------------+

+-------------------+
|            avg(_1)|
+-------------------+
|0.22423213650254825|
+-------------------+

